In [2]:
import json

In [3]:
def get_data(filename='data.json'):
    with open(filename, 'r') as f:
        return json.load(f)

In [4]:
def save_data(data, filename='data.json'):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

In [5]:
def generate_keys():
    data = get_data()

    data = {f"{i['buffer']};{i['pieces']};{i['alg']}": i for i in data}

    save_data(data)

In [6]:
def only_latest_algs(data):
    return {k: v for k, v in data.items() if v['latest_alg']}

In [7]:
def keys_with_given_buffer_targets_pair(data, buffer, targets):
    return [k for k, v in data.items() if [buffer, targets] == k.split(';')[:2]]

In [8]:
def is_key_in_dict(data, key):
    return key in data

In [9]:
def get_sheet_content(filename):
    result = []
    with open(filename) as f:
        for i in f:
            result.append(i.strip().split(';'))

    return result

In [10]:
VALID_CHARS = " UDFBRLMESudfbrlw'/:,2xyz"

def clean_alg_entry(alg):
    # filtering out invalid chars
    alg = ''.join([i for i in alg if i in VALID_CHARS])

    # cleaning multiple spaces 
    alg = ' '.join(alg.split())

    # cleaning whitespaces before , : '
    i = 0
    while i < len(alg):
        if alg[i] in "':," and i > 0 and alg[i - 1] == " ":
            alg = alg[:i-1] + alg[i:]
        else:
            i += 1

    return alg

In [11]:
alg = "[Ra    :  R  (1E ' 6  R    '&&   , %9 U       ']"
clean_alg_entry(alg)

"R: R E' R', U'"

In [12]:
def grid_to_alg_list(grid):
    result = []
    for i in range(1, len(grid)):
        for j in range(1, len(grid)):
            if grid[i][j]:
                alg = clean_alg_entry(grid[i][j])
                key = ";".join([grid[0][0], f'{grid[0][j]} {grid[i][0]}', alg])
                result.append(key)
    return result

In [13]:
def new_record_from_key(key):
    key = key.split(';')

    return {
        'buffer': key[0],
        'targets': key[1],
        'alg': key[2],
        'results': [],
        'latest': True
    }

In [14]:
def update_algs():
    data = get_data()
    algs = get_sheet_content('algs.csv')
    algs_list = grid_to_alg_list(algs)

    for alg in algs_list:
        decoded_key = alg.split(';')
        existing_algs = keys_with_given_buffer_targets_pair(data, decoded_key[0], decoded_key[1])

        for k in existing_algs:
            data[k]['latest'] = False

        if alg in data:
            data[alg]['latest'] = True
        else:
            data[alg] = new_record_from_key(alg)

    return data

In [15]:
from random import shuffle

def get_n_random_keys(data, n):
    data = {k: v for k, v in data.items() if v['latest']}
    if n > len(data):
        n = len(data)
    keys = list(data)
    shuffle(keys)
    return keys[:n]    

In [16]:
def display_record(alg):
    if 'word' in alg:
        print(alg['word'])
    elif 'lp' in alg:
        print(alg['lp'])
    else:
        print(alg['targets'])

In [17]:
from datetime import datetime

def measure_time():
    start = datetime.now()
    input()
    end = datetime.now()
    delta = (end - start).total_seconds()
    return round(delta, 2)

In [18]:
def get_response():
    print()
    response = input('Type Y to save results')
    return response == 'Y'

In [19]:
def add_entries(data, results):
    for k, v in results.items():
        data[k]['results'].append(v)

    save_data(data)

In [20]:
def run_game(n):
    results_dict = dict()
    data = get_data()
    algs = get_n_random_keys(data, n)
    for alg in algs:
        display_record(data[alg])
        exec_time = measure_time()
        results_dict[alg] = exec_time

    print('Results to be saved:')
    for k, v in results_dict.items():
        print(k.split(';')[1], v)

    if get_response():
        add_entries(data, results_dict)
    

In [21]:
def get_words_dict(grid):
    result = dict()
    for i in range(1, len(grid)):
        for j in range(1, len(grid[0])):
            if grid[i][j]:
                key = f'{grid[0][j]} {grid[i][0]}'
                result[key] = grid[i][j]
    return result

In [22]:
words = get_sheet_content('memo.csv')
get_words_dict(words)

{'UL UB': 'Bat',
 'UR UB': 'Czas',
 'UB UL': 'Arbuz',
 'UR UL': 'Cable',
 'UB UR': 'Acid',
 'UL UR': 'Baca'}

In [23]:
def update_words():
    data = get_data()
    words = get_sheet_content('memo.csv')
    words_dict = get_words_dict(words)

    for k, v in data.items():
        if v['targets'] in words_dict:
            data[k]['word'] = words_dict[v['targets']]

    return data

In [30]:
def get_lps_dict(grid):
    return {i[0]: i[1] for i in grid}

In [31]:
def update_LPs():
    data = get_data()
    lps = get_sheet_content('LP.csv')
    lps_dict = get_lps_dict(lps)

    for k, v in data.items():
        try:
            targets = v['targets'].split()
            data[k]['lp'] = ''.join([lps_dict[i] for i in targets])
        except KeyError:
            pass
    
    return data

In [32]:
update_LPs()

{"UF;UB UL;U' M2 U: M, U2": {'lp': 'AB',
  'alg': "U' M2 U: M, U2",
  'latest': True,
  'targets': 'UB UL',
  'buffer': 'UF',
  'word': 'Arbuz',
  'results': [0.24, 0.32, 0.31, 0.77, 4.96, 1.45]},
 "UF;UB UR;U M2 U': M, U2": {'lp': 'AC',
  'alg': "U M2 U': M, U2",
  'latest': True,
  'targets': 'UB UR',
  'buffer': 'UF',
  'word': 'Acid',
  'results': [0.54, 0.12, 0.41, 4.06, 0.54, 2.7, 1.33]},
 "UF;UB UR;R2 U': S, R2": {'lp': 'AC',
  'alg': "R2 U': S, R2",
  'latest': False,
  'targets': 'UB UR',
  'buffer': 'UF',
  'word': 'Acid',
  'results': [0.54, 0.27, 0.16]},
 "UF;UL UB;U' M2 U: U2, M": {'buffer': 'UF',
  'targets': 'UL UB',
  'alg': "U' M2 U: U2, M",
  'results': [0.96, 1.45],
  'latest': True,
  'word': 'Bat',
  'lp': 'BA'},
 "UF;UR UB;U M2 U': U2, M": {'buffer': 'UF',
  'targets': 'UR UB',
  'alg': "U M2 U': U2, M",
  'results': [0.78, 4.8, 2.71],
  'latest': True,
  'word': 'Czas',
  'lp': 'CA'},
 'UF;UR UL;M2 U: M, U2': {'buffer': 'UF',
  'targets': 'UR UL',
  'alg': 'M2 U:

In [33]:
data = update_words()
save_data(data, 'data.json')

In [36]:
run_game(5)

Bat
Acid
Baca
Czas
Arbuz
Results to be saved:
UL UB 1.45
UB UR 1.33
UL UR 1.5
UR UB 2.71
UB UL 1.45



In [133]:
data = get_data()
algs = get_n_random_keys(data, 3)
for k in algs:
    display_record(data[k])
    print()

Arbuz

UR UL

UR UB



In [134]:
measure_time()

 3


1.31

In [71]:
grid = get_sheet_content()

In [92]:
a = [1,2,3,4]
a[:3]

[1, 2, 3]

In [73]:
grid_to_alg_list(grid)

["UF;UL UB;U' M2 U: U2, M",
 "UF;UR UB;U M2 U': U2, M",
 "UF;UB UL;U' M2 U: M, U2",
 'UF;UR UL;M2 U: M, U2',
 "UF;UB UR;U M2 U': M, U2",
 "UF;UL UR;M2 U': M, U2"]

In [41]:
is_key_in_dict(data, "UF;UB UL;U' M2 U: M, U2")

True

In [35]:
count_buffer_targets(data, 'UF;UB UR')

2

In [38]:
data

{"UF;UB UL;U' M2 U: M, U2": {'lp': 'AB',
  'alg': "U' M2 U: M, U2",
  'latest_alg': True,
  'pieces': 'UB UL',
  'buffer': 'UF',
  'word': 'Arbuz',
  'results': [0.24, 0.32, 0.31]},
 "UF;UB UR;U M2 U': M, U2": {'lp': 'AC',
  'alg': "U M2 U': M, U2",
  'latest_alg': False,
  'pieces': 'UB UR',
  'buffer': 'UF',
  'word': 'Acid',
  'results': [0.54, 0.12, 0.41]},
 "UF;UB UR;R2 U': S, R2": {'lp': 'AC',
  'alg': "R2 U': S, R2",
  'latest_alg': True,
  'pieces': 'UB UR',
  'buffer': 'UF',
  'word': 'Acid',
  'results': [0.54, 0.27, 0.16]}}